In [2]:
import os,warnings
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import math

from netCDF4 import Dataset,num2date
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY, DAILY
from scipy import stats
from tqdm import tqdm
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter
from matplotlib.ticker import MultipleLocator

from mpl_toolkits.basemap import Basemap,maskoceans,interp,shiftgrid

warnings.simplefilter('ignore') # Turns off warnings

In [3]:
ENSO = {}
ENSO['year'],ENSO['month'],ENSO['index'] = np.loadtxt('ENSO.txt',skiprows=3,unpack=True)
ENSO['date'] = np.asarray([datetime(int(y),int(m),1) for y,m in zip(ENSO['year'],ENSO['month'])])

In [4]:
ncFile = 'onset.wet.season.GPCP.1979-2017.nc'
onsetFile = xr.open_dataset(ncFile, decode_times=False)

##starts in 1979

lon = onsetFile.lon.values
lat = onsetFile.lat.values
lev = onsetFile.lev.values

##northern great plains lon/lat
##112 w to 97 w
##40 to 48 north

ilat = np.where( (lat>=40) & (lat<=50) )[0]
ilon = np.where( (lon>=245) & (lon<=265) )[0]

plainsLat,plainsLon = np.meshgrid(lat[ilat],lon[ilon])

year = onsetFile.time.values
year = np.arange(1979, 2018, 1)

######### pulling out only the given lat/lon for pentad before doing the averaging


subset=onsetFile.sel(lat=slice(40,50), lon=slice(245,265))
subset=subset.load()

#print(subset)

pentad = subset['pentad']
print(pentad.shape)

averagePentad = np.nanmean(pentad, axis = 1) ## i want one value for each year for the domain
averagePentad = np.nanmean(averagePentad, axis = 1)
averagePentad = np.nanmean(averagePentad, axis = 1)

print(averagePentad)

onsetFile

(39, 1, 5, 9)
[21.416666 26.794443 24.477777 25.45     25.040741 24.862963 24.57963
 22.144445 24.459259 29.190742 27.535185 22.155556 21.537037 26.592592
 25.31852  26.116667 22.29074  25.514816 26.61111  21.937038 20.472223
 23.39074  26.214817 27.868519 23.049997 26.442593 25.607407 23.722221
 21.983334 27.346296 25.20926  21.051853 25.298147 21.92037  27.996298
 21.533333 26.181482 25.512962 23.605556]


<xarray.Dataset>
Dimensions:  (time: 39, lev: 1, lat: 72, lon: 144)
Coordinates:
  * time     (time) int32 0 1 2 3 4 5 6 7 8 9 ... 29 30 31 32 33 34 35 36 37 38
  * lev      (lev) int32 1000
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    pentad   (time, lev, lat, lon) float32 ...
    month    (time, lev, lat, lon) float32 ...
    year     (time, lev, lat, lon) float32 ...

In [11]:
ncFile = '500hpa.geopotential.heights.nc'
heightsFile = xr.open_dataset(ncFile)

heightsYear = heightsFile.time.dt.year
heightsYear = heightsFile.time.where((heightsFile.time.dt.year<=2017) & (heightsFile.time.dt.year>=1979), drop=True).values
heightsMonth = heightsFile.time.dt.month

heightsubset=heightsFile.sel(latitude=slice(50,40), longitude=slice(245,265), expver = 1)
heightsubset=heightsubset.load()

#print(subset)

heights = heightsubset['z']

print(heights.shape)
print(heightsYear)
print(heightsMonth)

heightsFile

(519, 41, 81)
['1979-01-01T00:00:00.000000000' '1979-02-01T00:00:00.000000000'
 '1979-03-01T00:00:00.000000000' '1979-04-01T00:00:00.000000000'
 '1979-05-01T00:00:00.000000000' '1979-06-01T00:00:00.000000000'
 '1979-07-01T00:00:00.000000000' '1979-08-01T00:00:00.000000000'
 '1979-09-01T00:00:00.000000000' '1979-10-01T00:00:00.000000000'
 '1979-11-01T00:00:00.000000000' '1979-12-01T00:00:00.000000000'
 '1980-01-01T00:00:00.000000000' '1980-02-01T00:00:00.000000000'
 '1980-03-01T00:00:00.000000000' '1980-04-01T00:00:00.000000000'
 '1980-05-01T00:00:00.000000000' '1980-06-01T00:00:00.000000000'
 '1980-07-01T00:00:00.000000000' '1980-08-01T00:00:00.000000000'
 '1980-09-01T00:00:00.000000000' '1980-10-01T00:00:00.000000000'
 '1980-11-01T00:00:00.000000000' '1980-12-01T00:00:00.000000000'
 '1981-01-01T00:00:00.000000000' '1981-02-01T00:00:00.000000000'
 '1981-03-01T00:00:00.000000000' '1981-04-01T00:00:00.000000000'
 '1981-05-01T00:00:00.000000000' '1981-06-01T00:00:00.000000000'
 '1981-07-0

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, expver: 2, time: 519)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2022-03-01
Data variables:
    z          (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-04-18 20:46:51 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

## Detrend and Standardize the Yearly Average Pentad

In [12]:
# detrend and standardize

t = np.arange(year.shape[0]) # An array for time spanning the length of the index
y = averagePentad ##
E = np.ones((t.size,2))*np.nan
E[:,0] = t # First column of your model E
E[:,1] = 1 # Second column of your model E
ETEinv = np.linalg.inv(E.T@E)
xhat = ETEinv@E.T@y
trend = E@xhat
detrend = y - trend

mean = detrend.mean()
std = detrend.std()
standardPentad = (detrend - mean) / std

print(standardPentad)

[-1.49678681  0.94023919 -0.1045099   0.33898025  0.15734216  0.08045067
 -0.04420481 -1.14258427 -0.09156507  2.05300971  1.30741542 -1.12334046
 -1.39966934  0.89154965  0.31857928  0.6832991  -1.04439888  0.41806681
  0.91769977 -1.19378798 -1.85307031 -0.52887259  0.75259119  1.50445388
 -0.6723979   0.86632308  0.49195118 -0.35755079 -1.14085241  1.28947001
  0.32600455 -1.55168985  0.37333421 -1.15157328  1.60136948 -1.31960148
  0.78726426  0.4883096  -0.37124731]


## Do Geopotential Height Anomalies

In [14]:
heightAnom = np.ones_like(heights)*np.nan # Makes an array the same shape as slpData with ones everywhere.
                                       # Then, multiply by NaN to make the array full of NaN values.

for imon in range(1,13):
    climatology = np.where( (heightsFile.time.dt.year>=1979) & (heightsFile.time.dt.year<=2000) & (heightsMonth==imon) )[0]
    x = np.where(heightsMonth==imon)[0]
    
    heightAnom[x,:,:] = heights[x,:,:] - np.nanmean(heights[climatology,:,:],0)
    
print(heightAnom.shape)

(519, 41, 81)


In [15]:
## detrend the height anomaly

t = np.arange(heightAnom.shape[0])
y = heightAnom.copy()
y = y.reshape(heightAnom.shape[0], heightAnom.shape[1]*heightAnom.shape[2])
E = np.ones((t.size,2))*np.nan
E[:,0] = t
E[:,1] = 1
ETEinv = np.linalg.inv(E.T@E)
xhat = ETEinv@E.T@y
trend = E@xhat
heightDetrend = y - trend
heightDetrend.shape

(519, 3321)

# COMPUTE THE LINEAR REGRESSION OF HEIGHT ONTO PENTADS

In [16]:
T,J,I = heightAnom.shape

y = heightDetrend.copy() #A copy of the detrended SLP anomalies, dimensioned time x space.

E = np.ones((T,2))
E[:,0] = standardPentad # First column of your model E
E[:,1] = 1 # Second column of your model E

xhat = np.dot(np.dot(np.linalg.inv(np.dot(E.T,E)),E.T),y)  # The least-squares best-fit parameters.

#=======================================================================
# Compute the "best fit" for SLP using the linear regression model (i.e., yhat=E*xhat)
#=======================================================================
yhat = E@xhat 

#=======================================================================
# Regression coefficients as a function of space (lon x lat). 
# This will be the slope parameter from xhat (i.e., the coefficient "a" from the regression equation above)
#=======================================================================
regressHeights = xhat[0,:]
regressHeights = regressHeights.reshape(J,I) #Reshape to lat x lon for plotting later.

print(regressHeights)
print(regressHeights.shape)

ValueError: could not broadcast input array from shape (39,) into shape (519,)

# ATTEMPT AT EOF

In [17]:
## steps from class
# remove seasonal cycle (don't need this)
# choose time period and domain (done)
# prep data by truncating to region (did this in loading it in)
# weight by sqrt of latitude?
# do svd
# regress the anomalies onto the og data
# calculate variance explained
# plot


In [34]:
ncFile = 'onset.wet.season.GPCP.1979-2017.nc'
onsetFile = xr.open_dataset(ncFile, decode_times=False)

##starts in 1979

lon = onsetFile.lon.values
lat = onsetFile.lat.values
lev = onsetFile.lev.values

##northern great plains lon/lat
##112 w to 97 w
##40 to 48 north

ilat = np.where( (lat>=20) & (lat<=50) )[0]
ilon = np.where( (lon>=230) & (lon<=300) )[0]

plotLat,plotLon = np.meshgrid(lat[ilat],lon[ilon])

year = onsetFile.time.values #1979 = 0
#year = np.arange(1979, 2018, 1)

######### pulling out only the given lat/lon for pentad before doing the averagin

pentad = onsetFile.pentad.values
pentadSubset = pentad[:,:,ilat,:][:,:,:,ilon][:,0,:,:]


print(pentad.shape)
print(pentadSubset.shape)

onsetFile

(39, 1, 72, 144)
(39, 13, 29)


<xarray.Dataset>
Dimensions:  (time: 39, lev: 1, lat: 72, lon: 144)
Coordinates:
  * time     (time) int32 0 1 2 3 4 5 6 7 8 9 ... 29 30 31 32 33 34 35 36 37 38
  * lev      (lev) int32 1000
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    pentad   (time, lev, lat, lon) float32 12.0 12.0 29.0 12.0 ... nan nan nan
    month    (time, lev, lat, lon) float32 ...
    year     (time, lev, lat, lon) float32 ...

In [35]:
weights = np.sqrt(np.cos(np.radians(ilat)))
weightedPentad = pentadSubset*weights[None,:,None]
print(weightedPentad.shape)

T,J,I = weightedPentad.shape
weightedPentadReshape = weightedPentad.reshape(T,J*I)
notNaN = ~np.isnan(weightedPentad[0,:])

A = weightedPentad[:,notNaN]
T,J,I = pentadSubset.shape
unweightedPentad = pentadSubset.reshape(T,J*I)

(39, 13, 29)


In [36]:
#SVD
T,N = A.shape
U,S,V = np.linalg.svd(A)

PCs = U
EOFs = A.T.dot(PCs)
eigval = S**2/N

LinAlgError: SVD did not converge